In [ ]:
# Script takes in credentials and flight info to check you in and email you your boarding pass
# Credit https://github.com/daveharmon for idea and code
from selenium import webdriver
from pyvirtualdisplay import Display
import arrow
import sys
from time import sleep
from bs4 import BeautifulSoup

In [ ]:
# Initializes selenium driver. Breaks after 10 tries to avoid infinite loop
def initializeDriver(count = 0):
    try:
        driver = webdriver.Chrome()
        driver.delete_all_cookies()
        driver.set_window_size("1920","1080")
        return driver
    except:
        if count < 10:
            count = count + 1
            time.sleep(3)
            return initializeDriver(count = count)
        else:
            return None

In [ ]:
# Need this for headless browsing
display = Display(visible=0, size=(1920, 1080)) 
display.start()

# Initialize driver
initializeDriver()

# Provide the starting page:
url = 'https://www.southwest.com/flight/retrieveCheckinDoc.html'

# Fill in credentials
# 'conf': 5 digit confirmation code
# 'fn': first name
# 'ln':last name
# 'flight_time': planned flight departure in YYYY-MM-DD HH:MM AM/PM format
# 'flight_tz': planned flight departure time zone (eg US/Eastern)
# 'email': email address to send boarding pass to
# Example:
# creds = {'conf':'12Q3WE', 'fn':'John','ln':'Smith', 'flight_time':'2017-08-25 06:15AM', 'flight_tz':'US/Eastern', 'email':'jsmith@sw.com'}

creds = {'conf':'', 'fn':'','ln':'', 'flight_time':'', 'flight_tz':'', 'email':''}

In [ ]:
# Method to combat any issues with driver timeoutes
try:driver.get(url)
except:
    driver = initializeDriver()
    driver.get(url)

time.sleep(1)

In [ ]:
# Enters information in
confirmation_num = driver.find_element_by_id("confirmationNumber")
first_name = driver.find_element_by_id("firstName")
last_name = driver.find_element_by_id("lastName")

confirmation_num.send_keys(creds['conf'])
first_name.send_keys(creds['fn'])
last_name.send_keys(creds['ln'])

In [ ]:
# Calculates the times necessary to determine when to check-in
current_time = arrow.now(creds['flight_tz'])
flight_time = arrow.get(creds['flight_time']).to(creds['flight_tz'])
deadline = flight_time.shift(days=-1)
deadline_minus_6s = deadline.shift(seconds=-6)

In [ ]:
# Checks every 5 seconds whether the current time is within 6 seconds of the deadline
# This reduces load on the kernal
while deadline_minus_6s > current_time:
    sys.stdout.write("Waiting for checkin time {0}\r".format(deadline.humanize()))
    sys.stdout.flush()
    sleep(5)
    current_time = arrow.now(creds['flight_tz'])
    
# Starts checking every cycle whether the current time has elapsed the deadline
while deadline > current_time:
    current_time  = arrow.now(creds['flight_tz'])

In [ ]:
# Check in
driver.find_element_by_id("submitButton").click()

In [ ]:
# This re-tries the submit if the script clicked submit too early
oops = driver.find_element_by_class_name("oopsError_message")
while oops.is_displayed():
    submit = driver.find_element_by_id("submitButton")
    while not submit.is_displayed():
        print "error displayed"
    submit.click()
    oops = driver.find_element_by_class_name("oopsError_message")

In [ ]:
# Once checked in, click the second confirm button to get the actual boarding number and option to email boarding pass
printDocs = driver.find_element_by_id('printDocumentsButton')
while not printDocs.is_displayed():
    print
printDocs.click()

In [ ]:
# Determines boarding position
soup = BeautifulSoup(driver.page_source, 'lxml')
subset = soup.find_all(class_='boardingInfo')
bp = ''

for x in subset:
    bp = bp + x.text

print bp

In [ ]:
# Emails boarding pass
driver.find_element_by_id("optionEmail").click()
email_box = driver.find_element_by_id("emailAddress")
email_box.send_keys(creds['email'])
driver.find_element_by_id("checkin_button").click()